In [1]:
from skdata.mnist.views import OfficialVectorClassification
from tqdm import tqdm
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
import time

from itertools import *
%matplotlib inline

In [2]:
# Generate Audio Data
def generate_audio_wave(frequency=440.0, framerate=44100, amplitude=0.5):
    i = 0
    while True:
        yield float(amplitude) + math.cos(2.0*math.pi*float(frequency)*(float(i)/float(framerate)))
        i += 1

In [3]:
def generate_seq(seq_leng):
    
    # Generate audio wave
    wave = generate_audio_wave(amplitude=1)
    
    # Generate function values
    data_seq = []
    for i in range(seq_leng):
        data_seq.append(wave.next())
    return data_seq

In [4]:
def generate_window(data, n_int, window_size):

    input_data=[]
    output_data=[]
    
    for i in range(0, n_int - window_size, 1):
        seq_in = data[i:i + window_size]
        seq_out = data[i + window_size]
        input_data.append(seq_in)
        output_data.append(seq_out)
    return input_data, output_data

In [5]:
data = generate_seq(100)

# Input, Output
input_data, output_data = generate_window(data, 100, 10)

In [ ]:
writer = tf.python_io.TFRecordWriter("sin.tfrecords")

In [ ]:
for i in tqdm(range(len(input_data))):
    features = input_data[i]
    label = output_data[i]
    # construct the Example proto boject
    example = tf.train.Example(
        # Example contains a Features proto object
        features=tf.train.Features(
          # Features contains a map of string to Feature proto objects
          feature={
            # A Feature contains one of either a float_list,
            # float_list, or bytes_list
            'label': tf.train.Feature(
                float_list=tf.train.FloatList(value=[label])),
            'seq': tf.train.Feature(
                float_list=tf.train.FloatList(value=features)),
    }))
    
    # use the proto object to serialize the example to a string
    serialized = example.SerializeToString()
    # write the serialized object to disk
    writer.write(serialized)

In [6]:
# Network Parameters
num_input = 10 # MNIST data input (img shape: 28*28)
num_hidden = 5 # hidden layer num of features

# Parameters
learning_rate = 0.001

In [7]:
filename_queue = tf.train.string_input_producer(["sin.tfrecords"], num_epochs=None)

In [8]:
reader = tf.TFRecordReader()

In [9]:
_, serialized_example = reader.read(filename_queue)

In [10]:
features = tf.parse_single_example(
        serialized_example,
        features={
            # We know the length of both fields. If not the
            # tf.VarLenFeature could be used
            'label': tf.FixedLenFeature([1], tf.float32),
            'seq': tf.FixedLenFeature([10], tf.float32)
        })

In [11]:
label = features['label']
seq = features['seq']

In [12]:
print seq.get_shape()
print label.get_shape()

(10,)
(1,)


In [13]:
# groups examples into batches randomly
seq_batch, labels_batch = tf.train.shuffle_batch(
    [seq, label], batch_size=num_hidden,
    capacity=2000,
    min_after_dequeue=1000)

In [14]:
print seq_batch.get_shape()
print labels_batch.get_shape()

(5, 10)
(5, 1)


In [15]:
seq_batch = tf.reshape(seq_batch, [num_hidden, num_input, 1])
labels_batch = tf.reshape(labels_batch, [num_hidden, 1])

In [16]:
print seq_batch.get_shape()
print labels_batch.get_shape()

(5, 10, 1)
(5, 1)


In [17]:
cell = tf.nn.rnn_cell.LSTMCell(num_hidden, state_is_tuple=True)
val, state = tf.nn.dynamic_rnn(cell, seq_batch, dtype=tf.float32)

In [18]:
val = tf.transpose(val, [1, 0, 2])
last = tf.gather(val, int(val.get_shape()[0]) - 1)

weight = tf.Variable(tf.truncated_normal([num_hidden, 1]))
bias = tf.Variable(tf.truncated_normal([1]))

# Functions definition 
pred = tf.matmul(last, weight) + bias

# Define loss and optimizer
cost = tf.sqrt(tf.reduce_mean(tf.square(labels_batch - pred)), name='RMSE')
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(labels_batch, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [19]:
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)
tf.train.start_queue_runners(sess=sess)

[<Thread(Thread-4, started daemon 123145332535296)>,
 <Thread(Thread-5, started daemon 123145336741888)>]

In [21]:
for i in range(1000):
  # pass it in through the feed_dict
  _, loss_val = sess.run([optimizer, cost])
  print loss_val

0.568455
0.423495
0.597345
0.481124
0.577397
0.595392
0.582464
0.425061
0.618704
0.589665
0.575004
0.500331
0.440581
0.350324
0.601807
0.627045
0.652415
0.678657
0.627612
0.598391
0.597733
0.649638
0.655104
0.550912
0.435636
0.447284
0.522478
0.566333
0.524137
0.422922
0.499264
0.443362
0.36989
0.495208
0.552446
0.349727
0.471909
0.284394
0.378921
0.338938
0.473509
0.490843
0.460219
0.32576
0.558713
0.548395
0.503338
0.447485
0.452337
0.404308
0.553269
0.41995
0.406808
0.320705
0.537165
0.404499
0.263595
0.406095
0.330639
0.407099
0.406621
0.346055
0.389151
0.337012
0.264485
0.41394
0.427803
0.28744
0.311523
0.295738
0.30357
0.452755
0.328746
0.34285
0.345744
0.219422
0.334222
0.241489
0.290964
0.426807
0.371165
0.448615
0.304283
0.360129
0.333859
0.396367
0.255257
0.327857
0.26092
0.345118
0.365282
0.335651
0.305021
0.373283
0.393642
0.324472
0.272494
0.257371
0.337004
0.367987
0.214414
0.278464
0.302425
0.270965
0.362138
0.234596
0.350166
0.335237
0.275914
0.291486
0.233908
0.260436


In [22]:
input_data[1]

[1.998035664431685,
 1.99215037495519,
 1.9823672529374121,
 1.9687247330474484,
 1.951276412259422,
 1.9300908392875948,
 1.9052512452810149,
 1.876855216835701,
 1.8450143126090013,
 1.8098536250423152]

In [23]:
output_data[1]

1.7715112889140305

In [24]:
# Train predicted
test = np.array(input_data[:5])
test = test.reshape(5, 10, 1)
pred = sess.run(pred ,{seq_batch: test})
print pred

[[ 1.79789829]
 [ 1.76347733]
 [ 1.72545743]
 [ 1.68392062]
 [ 1.63899827]]


In [25]:
output_data[:5]

[1.8098536250423152,
 1.7715112889140305,
 1.7301379386534053,
 1.6858961165474338,
 1.6389596341656558]